<a href="https://colab.research.google.com/github/summerdevlin46/recruitment_test/blob/main/bsc_recruitment_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BSC Recruitment Test

## Setting up environment

In [20]:
  !git clone https://github.com/summerdevlin46/recruitment_test

fatal: destination path 'recruitment_test' already exists and is not an empty directory.


In [22]:
from huggingface_hub import login
from google.colab import userdata

access_token_read = userdata.get('HF_TOKEN')

login(token = access_token_read)

In [23]:
import json

# Load the JSON data
with open("recruitment_test/data/articles.json", "r") as file:
    data = json.load(file)

## Preprocessing JSON Files

I removed any entries that contained null for each value, which left us with 36 entries containing usable data.

In [24]:
# Filter out the entries with all null values except for pmcid
filtered_articles = [entry for entry in data if any(value is not None for key, value in entry.items() if key != 'pmcid')]

# Print filtered data
print(json.dumps(filtered_articles, indent=4))

# Number of entries after filtering
#print("Number of entries after filtering:", len(filtered_articles))


[
    {
        "pmcid": "PMC10167034",
        "abstract": null,
        "intro": "1 Introduction Sepsis remains a leading cause of death in intensive care units (ICU), with a mortality rate of approximately 15-25% ( 1 \u2013 3 ). Vascular dysfunction, a feared complication of sepsis, can result in acute respiratory distress syndrome (ARDS), acute renal failure (ARF), and even mortality ( 4 , 5 ). However, existing prognostic tools are hindered by complexity and biases leading to imperfect predictive capabilities ( 6 \u2013 9 ). Further, the heterogeneity of sepsis at the individual patient level has also hindered therapeutic advances in the field ( 10 ). Decades of investigative efforts in the absence of sepsis endotyping have yielded no conclusively effective therapies to date. Understanding the heterogeneity of sepsis and subgrouping septic patients for personalized treatment may hold the promise of targeted sepsis therapy. Extracellular vesicles (EVs) are membranous nanovesicles f

## Setting up LLM and Prompting

I decided to use an Instruct model to do the data extraction pipeline. However, a NER pipline could also be used to complete this task

In [25]:
import torch
from transformers import pipeline

# Set Device to CPU if GPU is out of RAM

# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"   # Uses more GPU RAM
model_id = "meta-llama/Llama-3.2-3B-Instruct"        # Uses less GPU RAM
# model_id = "meta-llama/Llama-3.2-1B-Instruct"        # Uses the least GPU RAM
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map='auto',
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [26]:
# Function to process a single article and extract relevant information
def extract_information_from_article(article):
    # Combine relevant fields into a single text input
    article_content = (
        f"Abstract: {article.get('abstract', 'Not available')}\n"
        f"Introduction: {article.get('intro', 'Not available')}\n"
        f"Methods: {article.get('method', 'Not available')}\n"
        f"Subjects: {article.get('subjects', 'Not available')}\n"
        f"Results: {article.get('results', 'Not available')}\n"
        f"Discussion: {article.get('discussion', 'Not available')}\n"
        f"Conclusion: {article.get('conclusion', 'Not available')}\n"
    )

    # Define the prompt for extracting information
    # Prompt Engineering Reasoning:
    # After experimenting with multiple prompting approaches, this approach consistently produced the best results.
    #
    # Key Design Choices:
    # 1. Role Specification: Clearly defining the LLM's role as an "assistant trained to extract structured data"
    #    improves accuracy and consistency in extraction.
    # 2. JSON Format Enforcement: The output is explicitly structured in JSON, ensuring machine-readable and
    #    structured responses. If any information is missing, it is assigned a `null` value to maintain consistency.
    # 3. Field Definition: Each field is clearly specified, detailing the type of information expected.
    # 4. Flexibility in Study Design: Initially, a stricter JSON structure was enforced, but due to variability in
    #    research methodologies (e.g., some studies not having an experimental-control group setup), the prompt was
    #    adjusted to allow for different study designs. This ensures broader applicability across various scientific articles.
    # 5. Changes to patient origin: Seperated categories into study site and patient demographics to provide more accurate information.

    system_prompt = f"""
You are an assistant trained to extract structured data from the scientific article provided.
Extract the following information fields and return the output in JSON. If some information is missing, you respond with null.
1. disease_condition: Studied disease condition, health condition, disorder, syndrome, etc.
2. total_patients: Total number of patients
3. case_group_patients: Number of patients in experimental groups
4. control_group_patients: Number of patients in control groups
5. patient_sex: Sex of the patients
7. patient_demographics: demographics of the patients
8. study_site: location or place or hospital of study conducted
"""

    user_prompt = f""" Scientific article:
{article_content}
"""
    # Configure the messages for the LLM
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    outputs = pipe(
    messages,
    max_new_tokens=500,
)
    return outputs[0]["generated_text"][-1]['content']

## Processes the articles and saves extracted information

This is to provide the output in a similar JSON format as the original data provided.

In [27]:
# Formats extracted information into the correct format and return it in json format
def format_json(pmcid, content):

  extracted_info = {}
  assistant_content = content

  # Look for the part of the string between the triple backticks
  if "```" in assistant_content:
      # Split based on the '```' markers
      parts = assistant_content.split('```')
      if len(parts) > 1:
          # The second part should be the JSON content between the backticks
          json_content = parts[1].strip()  # Strip out unnecessary spaces/newlines
          try:
              # Parse the JSON content
              extracted_json = json.loads(json_content)
              extracted_info = extracted_json  # Store the extracted content
          except json.JSONDecodeError:
              # print(f"Error decoding JSON for pmcid: {pmcid}")
              return 0, None

  return 1, {'pmcid':pmcid, 'extracted_info':extracted_info}

This code processes a dataset of articles by extracting relevant information from each one, validating the extracted data, and retrying up to five times if errors are found. Successfully processed articles are stored in a results list, while errors are logged. Finally, the extracted information is saved as a JSON file at a specified location.

In [28]:
import json
from pathlib import Path

# Process all articles in the dataset
results = []

for article in filtered_articles:
    attempts = 0
    content = extract_information_from_article(article)

    # Checks for errors and sends back to LLM to reprocess if error is found
    while attempts < 5:
        check, formatted_extracted_info = format_json(article['pmcid'], content)
        if check == 1:
            results.append(formatted_extracted_info)
            print(f"PROCESSED article with PMCID: {article['pmcid']}")
            break
        # Re-extract content only if necessary
        content = extract_information_from_article(article)
        attempts += 1
    else:
        print(f"ERROR processing article with PMCID: {article['pmcid']}\nCONTENT:{content}\n")

# Save the results to a JSON file
output_path = Path("/content/recruitment_test/data/extracted_info.json")
output_path.write_text(json.dumps(results, indent=4))

print(f"Extraction complete. Results saved to {output_path}.")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


PROCESSED article with PMCID: PMC10167034


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


PROCESSED article with PMCID: PMC10191296


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

PROCESSED article with PMCID: PMC9381901


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


PROCESSED article with PMCID: PMC9529122


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


PROCESSED article with PMCID: PMC9616492
PROCESSED article with PMCID: PMC9697589
Extraction complete. Results saved to /content/recruitment_test/data/extracted_info.json.
